In [2]:
import os
mydir='/Users/l-c/Desktop/ABCluster-MacOSX/H2S5/h2s5-LM/'

In [3]:
from ase import Atoms, io
from ase import units
from math import cos, sin, pi
from ase.calculators.tip4p import TIP4P, epsilon0, sigma0, rOH, angleHOH
from ase.constraints import FixInternals
from ase.neb import NEB
from ase.optimize import MDMin, BFGS,LBFGS
from ase.optimize.basin_mul import BasinHoppingm
import pandas as pd
import numpy as np
from tools.h2s_cluster import h2scluster
from tools.h2s_cluster import add_const
import os
import argparse
import math 
from ase.optimize import LBFGS, FIRE


In [4]:
from os import listdir
def list_of_files(dir_name, suffix):
    return [f for f in listdir(dir_name) if f.endswith('.' + suffix)]
# load minima 
#ftraj = args.lm 

minima = list_of_files(mydir, 'xyz')
nminima = len(minima)
nminima

8

In [5]:
LM = {}
for mini in minima:
    LM[mini[:-4]]= io.read(mydir+mini)
for lm in LM.values():
    add_const(lm)


In [6]:

sep = math.floor(nminima/50)
minima=minima[-1::-sep]
nminima = len(minima)
print('# minima used: ', nminima)
LM = {}
for mini in minima:
    LM[mini[:-4]]= io.read(mydir+mini)
for lm in LM.values():
    add_const(lm)


ValueError: slice step cannot be zero

In [7]:
for lm in LM.values():
    dyn=LBFGS(lm)
    dyn.run(fmax=0.05)
    

       Step     Time          Energy         fmax
LBFGS:    0 00:42:37       -0.537369        0.0417
       Step     Time          Energy         fmax
LBFGS:    0 00:42:37       -0.558014        0.0462
       Step     Time          Energy         fmax
LBFGS:    0 00:42:37       -0.539447        0.0107
       Step     Time          Energy         fmax
LBFGS:    0 00:42:37       -0.553161        0.0283
       Step     Time          Energy         fmax
LBFGS:    0 00:42:37       -0.565390        0.0256
       Step     Time          Energy         fmax
LBFGS:    0 00:42:37       -0.573318        0.0410
       Step     Time          Energy         fmax
LBFGS:    0 00:42:37       -0.564135        0.0308
       Step     Time          Energy         fmax
LBFGS:    0 00:42:37       -0.606839        0.0278


In [11]:

import itertools
transition=pd.DataFrame(columns=['m1', 'm2', 'Ed', 'images'])
lmlist=LM.keys()
comb = list(itertools.product(lmlist, lmlist))
comb =[list(e) for e in set(frozenset(d) for d in comb)]
dcomb = []
for i in comb:
    if len(i)==2:
        dcomb.append(i)

dcomb=np.array(dcomb)
transition['m1'] = dcomb[:, 0]
#transition['m1']= transition['m1'].apply(lambda x: int(x))
transition['m2'] = dcomb[:, 1]
#transition['m2']= transition['m2'].apply(lambda x: int(x))
transition = transition[transition['m1']!=transition['m2']]
print(transition.head())
npdf=transition.values



########### neb 
import signal
class TimedOutExc(Exception):
    pass

def deadline(timeout, *args):
    def decorate(f):
        def handler(signum, frame):
            raise TimedOutExc()

        def new_f(*args):
            signal.signal(signal.SIGALRM, handler)
            signal.alarm(timeout)
            return f(*args)
            signal.alarm(0)

        new_f.__name__ = f.__name__
        return new_f
    return decorate

@deadline(50)
def optimize(neb="", name=''):
    optimizer = FIRE(neb, trajectory=name+'.traj')
    optimizer.run(fmax=100000)
    optimizer.run(fmax=50000)
    optimizer.run(fmax=10000)
    optimizer.run(fmax=5000)
    optimizer.run(fmax=1000)
    optimizer.run(fmax=500)
    optimizer.run(fmax=100)
    optimizer.run(fmax=50)
    optimizer.run(fmax=10)
    optimizer.run(fmax=0.5)
    optimizer.run(fmax=0.4)
    optimizer.run(fmax=0.3)
    optimizer.run(fmax=0.1)
    


  m1 m2   Ed images
0  3  6  NaN    NaN
1  1  6  NaN    NaN
2  7  4  NaN    NaN
3  3  7  NaN    NaN
4  1  2  NaN    NaN


In [12]:
min1='0'
min2='1'
images=[LM[min1]]
images += [LM[min1].copy() for i in range(5)]
images += [LM[min2]]
for image in images:
    add_const(image)
neb = NEB(images, parallel=True) 
neb.interpolate()
optimize(neb, 'check')

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 00:43:14   102948.541174* 1039463.1115
FIRE:    1 00:43:14     9702.857097*   82146.4200
FIRE:    2 00:43:15     1611.680527*    9377.1754
FIRE:    3 00:43:15     1387.514543*    6531.2534
FIRE:    4 00:43:15     1054.386203*    5007.6226
FIRE:    5 00:43:15      748.341760*    3486.6337
FIRE:    6 00:43:16      505.862106*    2163.8451
FIRE:    7 00:43:16      322.656267*    1300.8269
FIRE:    8 00:43:16      199.405731*     757.7417
FIRE:    9 00:43:16      123.573929*     422.4549
FIRE:   10 00:43:17       79.626325*     267.2295
FIRE:   11 00:43:17       54.652263*     198.9451
FIRE:   12 00:43:17       39.942489*     137.0314
FIRE:   13 00:43:17       30.416625*      88.1514
FIRE:   14 00:43:18       23.458725*      54.0017
FIRE:   15 00:43:18       17.908110*      42.4504
FIRE:   16 00:43:18       13.354825*      32.3151
FIRE:   17 00:43:18        9.690270*      23.3979
FI

In [13]:

from ase.neb import NEBTools
from ase.io import read
def saddlepoint(min1, min2, name):
    images=[LM[min1]]
    images += [LM[min1].copy() for i in range(5)]
    images += [LM[min2]]
    for image in images:
        add_const(image)
    neb = NEB(images, parallel=True) 
    try:
        neb.interpolate()
    except:
        try:
            print("try idpp interpolate")
            neb.interpolate(method="idpp")
        except:
            return
    optimize(neb, name)
ts = './h2s5_ts'
os.mkdir(ts) 
records=0
for line in npdf:
    records+=1
    try:
        line[3]=line[0]+'_'+line[1]
        saddlepoint(line[0], line[1], ts+ "/"+line[3])
    except:
        print(line[0], line[1], "fail")
        continue
transition=pd.DataFrame(npdf, columns=['m1', 'm2', 'Ed', 'images'])
transition.to_csv(ts+'.csv', index=False)



      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 00:43:53     5864.455442*   45910.9598
      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 00:43:54     5864.455442*   45910.9598
      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 00:43:54     5864.455442*   45910.9598
FIRE:    1 00:43:54     1009.140258*    5375.2610
FIRE:    2 00:43:54      488.755931*    1630.6019
FIRE:    3 00:43:55      451.300983*    1868.7694
FIRE:    4 00:43:55      402.940713*    1885.0971
FIRE:    5 00:43:55      296.092604*    1478.2273
FIRE:    6 00:43:55      175.787389*     899.6697
FIRE:    7 00:43:56       93.670613*     459.8899
FIRE:    8 00:43:56       78.591492*     208.2349
FIRE:    9 00:43:56       66.136646*     146.3194
FIRE:   10 00:43:56       53.773592*     127.6568
FIRE:   11 00:43:57       41.385573*     100.9385
FIRE: 

FIRE:  156 00:44:23       -0.236857*       0.3441
FIRE:  157 00:44:23       -0.237802*       0.3472
FIRE:  158 00:44:24       -0.238614*       0.3497
FIRE:  159 00:44:24       -0.239299*       0.3515
FIRE:  160 00:44:24       -0.239864*       0.3525
FIRE:  161 00:44:24       -0.240318*       0.3528
FIRE:  162 00:44:24       -0.240674*       0.3521
FIRE:  163 00:44:24       -0.240943*       0.3507
FIRE:  164 00:44:25       -0.241137*       0.3483
FIRE:  165 00:44:25       -0.241271*       0.3450
FIRE:  166 00:44:25       -0.241356*       0.3406
FIRE:  167 00:44:25       -0.241406*       0.3352
FIRE:  168 00:44:25       -0.241430*       0.3287
FIRE:  169 00:44:25       -0.241440*       0.3291
FIRE:  170 00:44:26       -0.241446*       0.3410
FIRE:  171 00:44:26       -0.241455*       0.3524
FIRE:  172 00:44:26       -0.241475*       0.3633
FIRE:  173 00:44:26       -0.241512*       0.3735
FIRE:  174 00:44:26       -0.241578*       0.3829
FIRE:  175 00:44:26       -0.241684*       0.3915


FIRE:   35 00:44:50        8.896198*      11.0014
FIRE:   36 00:44:50        7.477082*      10.5396
FIRE:   37 00:44:50        6.235761*       9.5141
FIRE:   38 00:44:51        5.156747*       8.1981
FIRE:   39 00:44:51        4.221212*       6.8404
FIRE:   40 00:44:51        3.411314*       5.5964
FIRE:   41 00:44:51        2.712590*       4.5301
FIRE:   42 00:44:51        2.115076*       3.6495
FIRE:   43 00:44:51        1.610492*       2.9360
FIRE:   44 00:44:52        1.310774*       2.3149
FIRE:   45 00:44:52        1.095841*       1.8749
FIRE:   46 00:44:52        0.916801*       1.5254
FIRE:   47 00:44:52        0.767717*       1.2467
FIRE:   48 00:44:52        0.643511*       1.0240
FIRE:   49 00:44:52        0.563867*       0.8705
FIRE:   50 00:44:53        0.515989*       0.7427
FIRE:   51 00:44:53        0.473421*       0.6277
FIRE:   52 00:44:53        0.435334*       0.5263
FIRE:   53 00:44:53        0.401050*       0.4378
FIRE:   54 00:44:53        0.370014*       0.3782


FIRE:  199 00:45:16       -0.043773*       0.2478
FIRE:  200 00:45:17       -0.040866*       0.2545
FIRE:  201 00:45:17       -0.038178*       0.2607
FIRE:  202 00:45:17       -0.035393*       0.2709
FIRE:  203 00:45:17       -0.032722*       0.2828
FIRE:  204 00:45:17       -0.030039*       0.2946
FIRE:  205 00:45:17       -0.027418*       0.3063
FIRE:  206 00:45:17       -0.024827*       0.3179
FIRE:  207 00:45:18       -0.022282*       0.3293
FIRE:  208 00:45:18       -0.019784*       0.3405
FIRE:  209 00:45:18       -0.017335*       0.3553
FIRE:  210 00:45:18       -0.014940*       0.3729
FIRE:  211 00:45:18       -0.012601*       0.3906
FIRE:  212 00:45:18       -0.010321*       0.4085
FIRE:  213 00:45:19       -0.008106*       0.4266
FIRE:  214 00:45:19       -0.005958*       0.4447
FIRE:  215 00:45:19       -0.003879*       0.4628
FIRE:  216 00:45:19       -0.001871*       0.4810
FIRE:  217 00:45:19        0.000067*       0.4992
FIRE:  218 00:45:19        0.001933*       0.5173


FIRE:   43 00:45:42       -0.101075*       0.2761
FIRE:   44 00:45:43       -0.106246*       0.2705
FIRE:   45 00:45:43       -0.112288*       0.2645
FIRE:   46 00:45:43       -0.119092*       0.2581
FIRE:   47 00:45:43       -0.126555*       0.2511
FIRE:   48 00:45:43       -0.134566*       0.2470
FIRE:   49 00:45:43       -0.143006*       0.2506
FIRE:   50 00:45:44       -0.151747*       0.2531
FIRE:   51 00:45:44       -0.160652*       0.2543
FIRE:   52 00:45:44       -0.169579*       0.2540
FIRE:   53 00:45:44       -0.178381*       0.2519
FIRE:   54 00:45:44       -0.186947*       0.2477
FIRE:   55 00:45:44       -0.195147*       0.2410
FIRE:   56 00:45:44       -0.202908*       0.2317
FIRE:   57 00:45:45       -0.210181*       0.2198
FIRE:   58 00:45:45       -0.216928*       0.2097
FIRE:   59 00:45:45       -0.223120*       0.1978
FIRE:   60 00:45:45       -0.228740*       0.1838
FIRE:   61 00:45:45       -0.233783*       0.1688
FIRE:   62 00:45:46       -0.238236*       0.1538


...

FIRE:   30 00:56:20        4.268413*       9.7054
FIRE:   31 00:56:20        3.488192*       7.5856
FIRE:   32 00:56:21        2.815390*       5.8047
FIRE:   33 00:56:21        2.245224*       4.3556
FIRE:   34 00:56:21        1.767800*       3.2098
FIRE:   35 00:56:21        1.371259*       2.3260
FIRE:   36 00:56:21        1.043778*       1.7863
FIRE:   37 00:56:21        0.774483*       1.4403
FIRE:   38 00:56:21        0.554024*       1.1532
FIRE:   39 00:56:22        0.374605*       0.9863
FIRE:   40 00:56:22        0.231284*       0.8429
FIRE:   41 00:56:22        0.149001*       0.7026
FIRE:   42 00:56:22        0.082555*       0.5743
FIRE:   43 00:56:22        0.063345*       0.5318
FIRE:   44 00:56:23        0.050258*       0.5067
FIRE:   45 00:56:23        0.037836*       0.4646
FIRE:   46 00:56:23        0.027833*       0.4133
FIRE:   47 00:56:23        0.018121*       0.3892
FIRE:   48 00:56:23        0.009737*       0.3874
FIRE:   49 00:56:24        0.001316*       0.3845


FIRE:  194 00:56:46       -0.167188*       0.4503
FIRE:  195 00:56:47       -0.169353*       0.4295
FIRE:  196 00:56:47       -0.171427*       0.4094
FIRE:  197 00:56:47       -0.173427*       0.3899
FIRE:  198 00:56:47       -0.175372*       0.3711
FIRE:  199 00:56:47       -0.177281*       0.3533
FIRE:  200 00:56:47       -0.179172*       0.3381
FIRE:  201 00:56:48       -0.181066*       0.3236
FIRE:  202 00:56:48       -0.182985*       0.3098
FIRE:  203 00:56:48       -0.184951*       0.2962
FIRE:  204 00:56:48       -0.186985*       0.2825
FIRE:  205 00:56:48       -0.189107*       0.2684
FIRE:  206 00:56:48       -0.191341*       0.2535
FIRE:  207 00:56:48       -0.193711*       0.2373
FIRE:  208 00:56:49       -0.196241*       0.2196
FIRE:  209 00:56:49       -0.198975*       0.2002
FIRE:  210 00:56:49       -0.201944*       0.1796
FIRE:  211 00:56:49       -0.205192*       0.1742
FIRE:  212 00:56:49       -0.208771*       0.1688
FIRE:  213 00:56:49       -0.211328*       0.4499


FIRE:   46 00:57:13       -0.186887*       0.3461
FIRE:   47 00:57:13       -0.193885*       0.3379
FIRE:   48 00:57:14       -0.200243*       0.3291
FIRE:   49 00:57:14       -0.206047*       0.3196
FIRE:   50 00:57:14       -0.211378*       0.3092
FIRE:   51 00:57:14       -0.216314*       0.2978
FIRE:   52 00:57:14       -0.220926*       0.2851
FIRE:   53 00:57:14       -0.225277*       0.2766
FIRE:   54 00:57:15       -0.229417*       0.2757
FIRE:   55 00:57:15       -0.233391*       0.2749
FIRE:   56 00:57:15       -0.237227*       0.2741
FIRE:   57 00:57:15       -0.240945*       0.2732
FIRE:   58 00:57:15       -0.244551*       0.2724
FIRE:   59 00:57:16       -0.248041*       0.2715
FIRE:   60 00:57:16       -0.251420*       0.2705
FIRE:   61 00:57:16       -0.254680*       0.2695
FIRE:   62 00:57:16       -0.257790*       0.2685
FIRE:   63 00:57:16       -0.260709*       0.2675
FIRE:   64 00:57:17       -0.263398*       0.2664
FIRE:   65 00:57:17       -0.265823*       0.2654


FIRE:  210 00:57:39       -0.440108*       0.2760
FIRE:  211 00:57:40       -0.442307*       0.2759
FIRE:  212 00:57:40       -0.444354*       0.2757
FIRE:  213 00:57:40       -0.446175*       0.2755
FIRE:  214 00:57:40       -0.447719*       0.2752
FIRE:  215 00:57:40       -0.448957*       0.2748
FIRE:  216 00:57:40       -0.449880*       0.2743
FIRE:  217 00:57:41       -0.450493*       0.2737
FIRE:  218 00:57:41       -0.449359*       0.2729
FIRE:  219 00:57:41       -0.446744*       0.2720
FIRE:  220 00:57:41       -0.444454*       0.2709
FIRE:  221 00:57:41       -0.442564*       0.2695
FIRE:  222 00:57:41       -0.441142*       0.2677
FIRE:  223 00:57:42       -0.440248*       0.2656
FIRE:  224 00:57:42       -0.439920*       0.2631
FIRE:  225 00:57:42       -0.440166*       0.2600
FIRE:  226 00:57:42       -0.440955*       0.2563
FIRE:  227 00:57:42       -0.442216*       0.2519
FIRE:  228 00:57:42       -0.443846*       0.2466
FIRE:  229 00:57:42       -0.445723*       0.2405


FIRE:   89 00:58:06       -0.231963*       0.2608
FIRE:   90 00:58:06       -0.236155*       0.2622
FIRE:   91 00:58:06       -0.239989*       0.2640
FIRE:   92 00:58:06       -0.243424*       0.2655
FIRE:   93 00:58:06       -0.246425*       0.2675
FIRE:   94 00:58:07       -0.248970*       0.2692
FIRE:   95 00:58:07       -0.251056*       0.2714
FIRE:   96 00:58:07       -0.252697*       0.2733
FIRE:   97 00:58:07       -0.253930*       0.2767
FIRE:   98 00:58:07       -0.254815*       0.2798
FIRE:   99 00:58:07       -0.255431*       0.2829
FIRE:  100 00:58:07       -0.255874*       0.2856
FIRE:  101 00:58:08       -0.256249*       0.2882
FIRE:  102 00:58:08       -0.256666*       0.2905
FIRE:  103 00:58:08       -0.257227*       0.2928
FIRE:  104 00:58:08       -0.258046*       0.2948
FIRE:  105 00:58:08       -0.259216*       0.2968
FIRE:  106 00:58:08       -0.260757*       0.2985
FIRE:  107 00:58:09       -0.262667*       0.3004
FIRE:  108 00:58:09       -0.264921*       0.3018
